# Astrometry

In [19]:
import h5py
import matplotlib.pyplot as plt
import numpy as np
from lenstronomy.Plots.model_plot import ModelPlot
import pickle 
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.io import fits

# define filters and system names
names = ['J0806+2006', 'J1442+4055', 'J1515+1511', 'J1620+1203', 'J2325-5229']
filter = 'F814W'

# table format:
# Lens System Name, RA_lens, Dec_lens, RA_im1, Dec_im1, Ra_im2, Dec_im2, Image Separation [arcsec]
print("\\begin{tabular}{lccccccc}")
print("\\hline")
print("Lens System & RA & DEC & RA$_{im1}$ & DEC$_{im1}$ & RA$_{im2}$ & DEC$_{im2}$ & Separation [arcsec]\\\\")
print("\\hline")

for i, name in enumerate(names):
    filename = f"../joint_modeling/{name}/{name}_joint.pkl"

    with open(filename, "rb") as f:
        loaded_data = pickle.load(f)

    kwargs_result = loaded_data["kwargs_result"]

    ra_lens = kwargs_result['kwargs_lens'][0]['center_x']
    dec_lens = kwargs_result['kwargs_lens'][0]['center_y']

    ra_im1 = kwargs_result['kwargs_ps'][0]['ra_image'][0] - ra_lens
    dec_im1 = kwargs_result['kwargs_ps'][0]['dec_image'][0] - dec_lens

    ra_im2 = kwargs_result['kwargs_ps'][0]['ra_image'][1] - ra_lens
    dec_im2 = kwargs_result['kwargs_ps'][0]['dec_image'][1] - dec_lens

    image_sep = np.sqrt((ra_im1 - ra_im2)**2 + (dec_im1 - dec_im2)**2)

    filename = f'../cutout_data/{name}/F814W/{name}_F814W_cutout.fits'
    hdul = fits.open(filename)
    header = hdul[0].header
    
    ra_deg = header['RA_TARG']
    dec_deg = header['DEC_TARG']
    
    coord = SkyCoord(ra=ra_deg*u.deg, dec=dec_deg*u.deg)
    
    # Keep RA and DEC to 5 significant digits
    RA = coord.ra.to_string(unit=u.hour, sep=':', precision=3)
    DEC = coord.dec.to_string(unit=u.deg, sep=':', precision=3)

    print(f"{name} & {RA} & {DEC} & {ra_im1:.3f} & {dec_im1:.3f} & {ra_im2:.3f} & {dec_im2:.3f} & {image_sep:.3f} \\\\")

print("\\hline")
print("\\end{tabular}")


\begin{tabular}{lccccccc}
\hline
Lens System & RA & DEC & RA$_{im1}$ & DEC$_{im1}$ & RA$_{im2}$ & DEC$_{im2}$ & Separation [arcsec]\\
\hline
J0806+2006 & 8:06:23.680 & 20:06:31.460 & -0.910 & 0.475 & 0.411 & -0.192 & 1.479 \\
J1442+4055 & 14:42:54.700 & 40:55:35.620 & -1.298 & 0.391 & 0.760 & -0.162 & 2.132 \\
J1515+1511 & 15:15:38.540 & 15:11:35.120 & -1.073 & 0.842 & 0.605 & -0.253 & 2.004 \\
J1620+1203 & 16:20:26.234 & 12:03:40.680 & -0.326 & -0.424 & 1.808 & 1.400 & 2.807 \\
J2325-5229 & 23:25:41.210 & -52:29:15.000 & -0.499 & 1.680 & 0.884 & -0.776 & 2.819 \\
\hline
\end{tabular}


# Magnitude Table

In [1]:
import h5py
import numpy as np
import os

# Define system names
names = ['J0806+2006', 
         'J1442+4055', 'J1515+1511', 'J1620+1203', 'J2325-5229']

def flux_ratio_and_sigma(fA, fB, sigmaA, sigmaB):
    """Compute flux ratio fA/fB with asymmetric errors."""
    ratio = fA / fB
    sigmaA_upper, sigmaA_lower = sigmaA
    sigmaB_upper, sigmaB_lower = sigmaB
    sigmaR_upper = ratio * np.sqrt((sigmaA_upper / fA)**2 + (sigmaB_lower / fB)**2)
    sigmaR_lower = ratio * np.sqrt((sigmaA_lower / fA)**2 + (sigmaB_upper / fB)**2)
    return ratio, sigmaR_upper, sigmaR_lower

def format_quantity(val, upper, lower, decimals=2):
    return f"${val:.{decimals}f}^{{+{upper:.{decimals}f}}}_{{-{lower:.{decimals}f}}}$"

def mag_sigma(flux, sigma_flux, zp_err_mag=0.018):
    sigma_flux = np.asarray(sigma_flux)
    sigma_upper = sigma_flux[0]
    sigma_lower = sigma_flux[1]
    zp_flux_err = 0.921 * flux * zp_err_mag  # Zeropoint mag error converted to flux error
    total_upper = np.sqrt(sigma_upper**2 + zp_flux_err**2)
    total_lower = np.sqrt(sigma_lower**2 + zp_flux_err**2)
    mag_err_upper = (2.5 / np.log(10)) * (total_upper / flux)
    mag_err_lower = (2.5 / np.log(10)) * (total_lower / flux)
    return mag_err_upper, mag_err_lower

def format_mag(mag, sigmas):
    return f"${mag:.2f}^{{+{sigmas[0]:.2f}}}_{{-{sigmas[1]:.2f}}}$"

# --- LaTeX header ---
print("\\begin{tabular*}{\\textwidth}{@{\\extracolsep{\\fill}}lccccccc}")
print("\\hline")
print("Lens System & Filter & Lens & Image A & Image B & Host (lensed) & Host (delensed) & $f_A/f_B$ \\\\")
print("\\hline")

# --- Loop over systems ---
for name in names:
    row_started = False
    # Change order here
    for filt in ["F160W", "F814W", "F475X"]:
        filename = f"../joint_modeling/{name}/{name}_photometry.hdf5"

        if os.path.exists(filename):
            with h5py.File(filename, "r") as f:
                if filt in f:
                    g = f[filt]

                    # --- Lens photometry ---
                    lens_mag = g["lens_mag"][()]
                    lens_flux = g["lens_flux"][()]
                    lens_flux_sigma = g["lens_flux_sigma"][()]
                    lens_mag_fmt = format_mag(lens_mag, mag_sigma(lens_flux, lens_flux_sigma))

                    # --- Image A ---
                    fA = g["image1_flux"][()]
                    sigmaA = g["image1_flux_sigma"][()]
                    magA = g["image1_mag"][()]
                    magA_fmt = format_mag(magA, mag_sigma(fA, sigmaA))

                    # --- Image B ---
                    fB = g["image2_flux"][()]
                    sigmaB = g["image2_flux_sigma"][()]
                    magB = g["image2_mag"][()]
                    magB_fmt = format_mag(magB, mag_sigma(fB, sigmaB))

                    # --- Flux ratio ---
                    ratio, r_up, r_low = flux_ratio_and_sigma(fA, fB, sigmaA, sigmaB)
                    flux_ratio_fmt = format_quantity(ratio, r_up, r_low, decimals=2)

                    # --- Host (lensed) ---
                    if "host_lensed_mag" in g:
                        host_lensed_mag = g["host_lensed_mag"][()]
                        if np.isnan(host_lensed_mag):
                            host_lensed_fmt = "N/A"
                        else:
                            host_lensed_flux = g["host_lensed_flux"][()]
                            host_lensed_flux_sigma = g["host_lensed_flux_sigma"][()]
                            host_lensed_fmt = format_mag(
                                host_lensed_mag, mag_sigma(host_lensed_flux, host_lensed_flux_sigma)
                            )
                    else:
                        host_lensed_fmt = "--"

                    # --- Host (delensed / intrinsic) ---
                    if "host_intrinsic_mag" in g:
                        host_intrinsic_mag = g["host_intrinsic_mag"][()]
                        if np.isnan(host_intrinsic_mag):
                            host_intrinsic_fmt = "N/A"
                        else:
                            host_intrinsic_flux = g["host_intrinsic_flux"][()]
                            host_intrinsic_flux_sigma = g["host_intrinsic_flux_sigma"][()]
                            host_intrinsic_fmt = format_mag(
                                host_intrinsic_mag, mag_sigma(host_intrinsic_flux, host_intrinsic_flux_sigma)
                            )
                    else:
                        host_intrinsic_fmt = "--"

                else:
                    lens_mag_fmt = magA_fmt = magB_fmt = flux_ratio_fmt = host_lensed_fmt = host_intrinsic_fmt = "--"
        else:
            lens_mag_fmt = magA_fmt = magB_fmt = flux_ratio_fmt = host_lensed_fmt = host_intrinsic_fmt = "--"

        # --- Print table row ---
        if not row_started:
            print(f"{name} & {filt} & {lens_mag_fmt} & {magA_fmt} & {magB_fmt} & {host_lensed_fmt} & {host_intrinsic_fmt} & {flux_ratio_fmt} \\\\")
            row_started = True
        else:
            print(f" & {filt} & {lens_mag_fmt} & {magA_fmt} & {magB_fmt} & {host_lensed_fmt} & {host_intrinsic_fmt} & {flux_ratio_fmt} \\\\")

    print("\\hline")

print("\\end{tabular*}")


\begin{tabular*}{\textwidth}{@{\extracolsep{\fill}}lccccccc}
\hline
Lens System & Filter & Lens & Image A & Image B & Host (lensed) & Host (delensed) & $f_A/f_B$ \\
\hline
J0806+2006 & F160W & $18.67^{+0.03}_{-0.03}$ & $18.79^{+0.03}_{-0.03}$ & $19.69^{+0.04}_{-0.04}$ & $20.54^{+0.12}_{-0.09}$ & $22.13^{+0.13}_{-0.10}$ & $2.29^{+0.10}_{-0.10}$ \\
 & F814W & $19.67^{+0.02}_{-0.02}$ & $19.26^{+0.02}_{-0.02}$ & $19.98^{+0.02}_{-0.02}$ & $23.30^{+0.09}_{-0.09}$ & $24.88^{+0.10}_{-0.09}$ & $1.93^{+0.01}_{-0.01}$ \\
 & F475X & $21.88^{+0.03}_{-0.03}$ & $20.29^{+0.02}_{-0.02}$ & $21.16^{+0.02}_{-0.02}$ & $23.53^{+0.04}_{-0.04}$ & $25.11^{+0.05}_{-0.05}$ & $2.22^{+0.01}_{-0.02}$ \\
\hline
J1442+4055 & F160W & $18.16^{+0.02}_{-0.02}$ & $18.01^{+0.02}_{-0.02}$ & $18.43^{+0.02}_{-0.02}$ & $19.32^{+0.03}_{-0.03}$ & $21.53^{+0.04}_{-0.04}$ & $1.47^{+0.02}_{-0.02}$ \\
 & F814W & $18.75^{+0.02}_{-0.02}$ & $18.22^{+0.02}_{-0.02}$ & $19.01^{+0.02}_{-0.02}$ & N/A & N/A & $2.08^{+0.01}_{-0.01}$ \\
 & F47

# Lens Light Parameter Table

In [1]:
import pickle
import numpy as np
import os

names = ['J0806+2006',
         'J1442+4055', 'J1515+1511', 'J1620+1203', 'J2325-5229']

def get_median_and_uncertainties(samples):
    median = np.percentile(samples, 50)
    lower = median - np.percentile(samples, 16)
    upper = np.percentile(samples, 84) - median
    return median, lower, upper

def format_val(med, lo, up, fmt=".2f"):
    return f"${med:{fmt}}^{{+{up:{fmt}}}}_{{-{lo:{fmt}}}}$"

def extract_profile(samples, param_names, param_mcmc):
    """Return R, n, q, phi chains for one profile."""
    idx = [param_mcmc.index(p) for p in param_names]
    r_chain = samples[:, idx[0]]
    n_chain = samples[:, idx[1]]
    e1_chain = samples[:, idx[2]]
    e2_chain = samples[:, idx[3]]
    e_chain = np.sqrt(e1_chain**2 + e2_chain**2)
    q_chain = (1 - e_chain) / (1 + e_chain)
    phi_chain = 0.5 * np.arctan2(e2_chain, e1_chain) * 180 / np.pi
    return r_chain, n_chain, q_chain, phi_chain

def summarize_chain(chain, fmt=".2f"):
    med, lo, up = get_median_and_uncertainties(chain)
    return format_val(med, lo, up, fmt)

# Map each filter to its corresponding lens light indices in chain_list
filter_map = {
    "F160W": [0, 1],
    "F814W": [2],
    "F475X": [3],
}

# --- LaTeX header ---
print("\\begin{tabular*}{\\textwidth}{@{\\extracolsep{\\fill}}lccccc}")
print("\\hline")
print("Lens System & Filter & $R_{\\text{S\\'ersic}}$ [arcsec] & "
      "$n_{\\text{S\\'ersic}}$ & $q$ & $\\phi_{\\text{ext}}$ [deg] \\\\")
print("\\hline")

# --- Loop over systems ---
for name in names:
    filename = f"../joint_modeling/{name}/{name}_joint.pkl"
    if not os.path.exists(filename):
        print(f"{name} & -- & -- & -- & -- & -- \\\\")
        print("\\hline")
        continue

    with open(filename, "rb") as f:
        loaded_data = pickle.load(f)

    chain_list = loaded_data.get("chain_list")
    sampler_type, samples, param_mcmc, dist_mcmc = chain_list[3]

    row_started = False
    for filt in ["F160W", "F814W", "F475X"]:
        prof_indices = filter_map.get(filt, [])
        if not prof_indices:
            continue

        for prof_idx in prof_indices:
            try:
                param_names = [
                    f"R_sersic_lens_light{prof_idx}",
                    f"n_sersic_lens_light{prof_idx}",
                    f"e1_lens_light{prof_idx}",
                    f"e2_lens_light{prof_idx}"
                ]

                r_chain, n_chain, q_chain, phi_chain = extract_profile(samples, param_names, param_mcmc)

                r_str = summarize_chain(r_chain)
                n_str = summarize_chain(n_chain)
                q_str = summarize_chain(q_chain)
                phi_str = summarize_chain(phi_chain, fmt=".1f")

                if not row_started:
                    print(f"{name} & {filt} & {r_str} & {n_str} & {q_str} & {phi_str} \\\\")
                    row_started = True
                else:
                    print(f" & {filt} & {r_str} & {n_str} & {q_str} & {phi_str} \\\\")
            except Exception:
                # Handle missing filter parameters gracefully
                if not row_started:
                    print(f"{name} & {filt} & -- & -- & -- & -- \\\\")
                    row_started = True
                else:
                    print(f" & {filt} & -- & -- & -- & -- \\\\")

    print("\\hline")

print("\\end{tabular*}")


\begin{tabular*}{\textwidth}{@{\extracolsep{\fill}}lccccc}
\hline
Lens System & Filter & $R_{\text{S\'ersic}}$ [arcsec] & $n_{\text{S\'ersic}}$ & $q$ & $\phi_{\text{ext}}$ [deg] \\
\hline
J0806+2006 & F160W & $0.31^{+0.03}_{-0.03}$ & $6.90^{+0.06}_{-0.09}$ & $0.89^{+0.02}_{-0.02}$ & $-77.1^{+4.1}_{-4.3}$ \\
 & F160W & $1.13^{+0.04}_{-0.04}$ & $1.47^{+0.20}_{-0.16}$ & $0.63^{+0.03}_{-0.02}$ & $40.9^{+2.1}_{-2.1}$ \\
 & F814W & $0.80^{+0.03}_{-0.03}$ & $5.26^{+0.12}_{-0.14}$ & $0.94^{+0.01}_{-0.01}$ & $79.4^{+5.0}_{-5.3}$ \\
 & F475X & $2.10^{+0.19}_{-0.20}$ & $5.98^{+0.17}_{-0.18}$ & $0.70^{+0.04}_{-0.03}$ & $-10.7^{+4.4}_{-4.1}$ \\
\hline
J1442+4055 & F160W & $0.24^{+0.03}_{-0.03}$ & $6.97^{+0.02}_{-0.03}$ & $0.81^{+0.01}_{-0.01}$ & $61.9^{+1.1}_{-1.1}$ \\
 & F160W & $0.15^{+0.01}_{-0.01}$ & $0.61^{+0.32}_{-0.30}$ & $0.85^{+0.01}_{-0.01}$ & $83.9^{+3.9}_{-171.8}$ \\
 & F814W & $1.35^{+0.02}_{-0.03}$ & $6.95^{+0.03}_{-0.05}$ & $0.82^{+0.00}_{-0.00}$ & $51.9^{+0.8}_{-0.8}$ \\
 & F475X & 

# Lens MASS

In [3]:
import pickle
import numpy as np
import os

names = ['J0806+2006',
         'J1442+4055', 'J1515+1511', 'J1620+1203', 'J2325-5229']

def get_median_and_uncertainties(samples):
    median = np.percentile(samples, 50)
    lower = median - np.percentile(samples, 16)
    upper = np.percentile(samples, 84) - median
    return median, lower, upper

# --- LaTeX table header ---
print("\\begin{tabular*}{\\textwidth}{@{\\extracolsep{\\fill}}lcccc}")
print("\\hline")
print("Lens System & $\\theta_E$ [arcsec] & $q_{\\text{mass}}$ & $\\phi_{\\text{mass}}$ [deg] & $\\gamma$ \\\\")
print("\\hline")

# --- Loop over systems ---
for name in names:
    filename = f"../joint_modeling/{name}/{name}_joint.pkl"
    if not os.path.exists(filename):
        print(f"{name} & -- & -- & -- & -- \\\\")
        print("\\hline")
        continue

    with open(filename, "rb") as f:
        loaded_data = pickle.load(f)

    chain_list = loaded_data.get('chain_list')
    sampler_type, samples_mcmc, param_mcmc, dist_mcmc = chain_list[3]

    # Parameters for the single lens mass model
    param_names_mass = ['theta_E_lens0', 'e1_lens0', 'e2_lens0', 'gamma1_lens1', 'gamma2_lens1']
    indices = [param_mcmc.index(p) for p in param_names_mass]

    theta_E_chain = samples_mcmc[:, indices[0]]
    e1_chain = samples_mcmc[:, indices[1]]
    e2_chain = samples_mcmc[:, indices[2]]
    gamma1_chain = samples_mcmc[:, indices[3]]
    gamma2_chain = samples_mcmc[:, indices[4]]

    # Derived quantities
    e_chain = np.sqrt(e1_chain**2 + e2_chain**2)
    q_mass_chain = (1 - e_chain) / (1 + e_chain)
    gamma_chain = np.sqrt(gamma1_chain**2 + gamma2_chain**2)
    phi_mass_chain = 0.5 * np.arctan2(e2_chain, e1_chain) * 180 / np.pi

    # Medians and uncertainties
    theta_E_median, theta_E_lower, theta_E_upper = get_median_and_uncertainties(theta_E_chain)
    q_mass_median, q_mass_lower, q_mass_upper = get_median_and_uncertainties(q_mass_chain)
    gamma_median, gamma_lower, gamma_upper = get_median_and_uncertainties(gamma_chain)
    phi_mass_median, phi_mass_lower, phi_mass_upper = get_median_and_uncertainties(phi_mass_chain)

    # Format LaTeX values
    theta_E = f"${theta_E_median:.2f}^{{+{theta_E_upper:.2f}}}_{{-{theta_E_lower:.2f}}}$"
    q_mass = f"${q_mass_median:.2f}^{{+{q_mass_upper:.2f}}}_{{-{q_mass_lower:.2f}}}$"
    gamma = f"${gamma_median:.2f}^{{+{gamma_upper:.2f}}}_{{-{gamma_lower:.2f}}}$"
    phi_mass = f"${phi_mass_median:.1f}^{{+{phi_mass_upper:.1f}}}_{{-{phi_mass_lower:.1f}}}$"

    # Print one row per system
    print(f"{name} & {theta_E} & {q_mass} & {phi_mass} & {gamma} \\\\")

print("\\end{tabular*}")


\begin{tabular*}{\textwidth}{@{\extracolsep{\fill}}lcccc}
\hline
Lens System & $\theta_E$ [arcsec] & $q_{\text{mass}}$ & $\phi_{\text{mass}}$ [deg] & $\gamma$ \\
\hline
J0806+2006 & $0.55^{+0.01}_{-0.01}$ & $0.64^{+0.03}_{-0.03}$ & $-70.7^{+4.6}_{-4.3}$ & $0.30^{+0.01}_{-0.01}$ \\
J1442+4055 & $1.06^{+0.01}_{-0.01}$ & $0.82^{+0.02}_{-0.02}$ & $-58.3^{+6.2}_{-5.0}$ & $0.08^{+0.01}_{-0.01}$ \\
J1515+1511 & $0.66^{+0.01}_{-0.01}$ & $0.61^{+0.02}_{-0.01}$ & $52.2^{+1.5}_{-2.4}$ & $0.31^{+0.00}_{-0.01}$ \\
J1620+1203 & $1.45^{+0.01}_{-0.02}$ & $0.87^{+0.03}_{-0.03}$ & $-49.1^{+10.4}_{-21.8}$ & $0.09^{+0.02}_{-0.03}$ \\
J2325-5229 & $1.50^{+0.00}_{-0.01}$ & $0.87^{+0.01}_{-0.07}$ & $-0.4^{+1.5}_{-2.3}$ & $0.04^{+0.00}_{-0.01}$ \\
\end{tabular*}


# Fermat Potential and Time Delays

In [6]:
import h5py
import numpy as np

names_f814w = ['J0806+2006', 'J1442+4055', 'J1620+1203', 'J2325-5229']

z_l = [0.573, 0.284, 0.398, 0.4]
z_s = [1.54, 2.593, 1.158, 2.739]

# LaTeX table header for F814W only
print("\\begin{table}[htb]")
print("\\centering")
print("\\caption{blah blah}")
print("\\label{tab:cosmo_results}")
print("\\begin{tabular}{lcccc}")
print("\\hline")
print("Lens System & $z_l$ & $z_s$ & $\\Delta \\Phi_{AB}$ [arcsec$\\^2$] & $\\Delta t_{AB}$ [days] \\\\")
print("\\hline")

def get_median_and_uncertainties(samples):
    median = np.percentile(samples, 50)
    lower = median - np.percentile(samples, 16)
    upper = np.percentile(samples, 84) - median
    return median, lower, upper

for i, name in enumerate(names_f814w):
    filename_f814w = f"../cutout_data/{name}/F814W/{name}_F814W_cosmo.hdf5"

    with h5py.File(filename_f814w, "r") as f:
            kwargs = {}
            for key in f:
                kwargs[key] = f[key][()]

    dphi_chain = kwargs['dphi_list']
    dt_chain = kwargs['dt_list']

    dphi_median, dphi_lower, dphi_upper = get_median_and_uncertainties(dphi_chain)
    dt_median, dt_lower, dt_upper = get_median_and_uncertainties(dt_chain)

    z_l_fmt = f"${z_l[i]}$"
    z_s_fmt = f"${z_s[i]}$"
    dphi_fmt = f"${dphi_median:.3f}^{{+{dphi_upper:.3f}}}_{{-{dphi_lower:.3f}}}$"
    dt_fmt = f"${dt_median:.1f}^{{+{dt_upper:.1f}}}_{{-{dt_lower:.1f}}}$"

    print(f"{name} & {z_l_fmt} & {z_s_fmt} & {dphi_fmt} & {dt_fmt} \\\\")

print("\\hline")
print("\\end{tabular}")
print("\\end{table}")


\begin{table}[htb]
\centering
\caption{blah blah}
\label{tab:cosmo_results}
\begin{tabular}{lcccc}
\hline
Lens System & $z_l$ & $z_s$ & $\Delta \Phi_{AB}$ [arcsec$\^2$] & $\Delta t_{AB}$ [days] \\
\hline
J0806+2006 & $0.573$ & $1.54$ & $-0.468^{+0.023}_{-0.010}$ & $-53.3^{+2.6}_{-1.2}$ \\
J1442+4055 & $0.284$ & $2.593$ & $-0.544^{+0.004}_{-0.006}$ & $-21.4^{+0.2}_{-0.2}$ \\
J1620+1203 & $0.398$ & $1.158$ & $2.453^{+0.071}_{-0.080}$ & $183.2^{+5.3}_{-6.0}$ \\
J2325-5229 & $0.4$ & $2.739$ & $-0.712^{+0.031}_{-0.024}$ & $-41.5^{+1.8}_{-1.4}$ \\
\hline
\end{tabular}
\end{table}
